In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from gradio_client import Client
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch
from langchain import PromptTemplate,  LLMChain
import replicate
import os

In [ ]:
pip install gradio_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random

# Create a list of 10,000 user IDs
user_ids = ['User{:04d}'.format(i) for i in range(1, 10001)]

# Generate random binary predictions (0 or 1) for churn
churn_predictions = [random.randint(0, 1) for _ in range(10000)]

# Create a DataFrame
df = pd.DataFrame({'UserID': user_ids, 'ChurnPrediction': churn_predictions})

# Display the DataFrame
print(df)

         UserID  ChurnPrediction
0      User0001                0
1      User0002                0
2      User0003                0
3      User0004                1
4      User0005                0
...         ...              ...
9995   User9996                1
9996   User9997                0
9997   User9998                0
9998   User9999                0
9999  User10000                1

[10000 rows x 2 columns]


In [ ]:
df['LoyatyTag'] = df['ChurnPrediction'].apply(lambda x: 1 if x == 1 else 0)


In [ ]:
pip install replicate

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Churn_Modelling - Churn_Modelling.csv')


In [4]:
test = df[['Surname', "Gender", "Geography", "CreditScore", "Geography","Age","Balance","NumOfProducts","HasCrCard", "EstimatedSalary","Exited"]]
test = test[test['Exited']==1]
test

,Surname,Gender,Geography,CreditScore,Geography,Age,Balance,NumOfProducts,HasCrCard,EstimatedSalary,Exited
0,Hargrave,Female,France,619,France,42,0.00,1,1,101348.90,1
2,Onio,Female,France,502,France,42,159660.80,3,1,113931.60,1
5,Chu,Male,Spain,645,Spain,44,113755.80,2,1,149756.70,1
7,Obinna,Female,Germany,376,Germany,29,115046.70,4,1,119346.90,1
16,Romeo,Male,Germany,653,Germany,58,132602.90,1,1,5097.67,1
...,...,...,...,...,...,...,...,...,...,...,...
9981,Burbidge,Male,Germany,498,Germany,42,152039.70,1,1,53445.17,1
9982,Griffin,Female,Germany,655,Germany,46,137145.10,1,1,115146.40,1
9991,Ajuluchukwu,Female,France,597,France,53,88381.21,1,1,69384.71,1
9997,Liu,Female,France,709,France,36,0.00,1,0,42085.58,1


Attempt 1: Using Replicate API to access Huggingface Llama2 Model for text generation


In [ ]:
pip install replicate

In [ ]:
import os

os.environ["REPLICATE_API_TOKEN"] = "API"

In [5]:
import pandas as pd
import random

num_rows = test.shape[0]

# Select 100 random row indices
random_row_indices = random.sample(range(num_rows), 10)

# Create a new DataFrame with the selected rows
selected_data = test.iloc[random_row_indices]

# Now, 'selected_data' contains only the 100 randomly selected rows

In [6]:
selected_data

,Surname,Gender,Geography,CreditScore,Geography,Age,Balance,NumOfProducts,HasCrCard,EstimatedSalary,Exited
7338,Smith,Female,Germany,708,Germany,54,145151.40,1,0,125311.20,1
6596,Tuan,Male,France,477,France,22,82559.42,2,0,163112.90,1
1092,Bergamaschi,Male,France,715,France,41,94267.90,1,0,152821.10,1
1786,Fang,Male,Spain,711,Spain,37,0.00,2,1,83978.86,1
5310,Tai,Male,Germany,622,Germany,42,115766.30,1,0,72155.85,1
1388,McIntyre,Female,France,630,France,40,118633.10,1,0,60032.46,1
9960,Lucciano,Female,Germany,795,Germany,33,104552.70,1,1,120853.80,1
484,George,Male,Spain,615,Spain,61,0.00,2,1,150227.90,1
2431,Yeh,Male,France,540,France,22,0.00,3,1,186233.30,1
3646,Thompson,Female,France,770,France,70,110738.90,1,1,22666.77,1


In [ ]:
c
pre_prompt = selected_data#"You are a helpful assistant. The data you have access to customer bank data, so you have their credit scores, geography (France, Spain, Germany), age, gender, Tenure (the time of bond with a company), balance, and numproducts (the number of products they own). You do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'."
prompt_input = "In the pre_promt I gave you a dataset containing 10 customer data. The data you have access to customer bank data, USE THIS DATA FOR THE OUTPUT. THIS IS NOT A CONVERSATION. Please come up with text that would appeal to the different types of people in the data for loyalty rewards, these customers in the data are prone to leaving the company, so if they have a good credit score give them a good offer of an interest rate for 2 years between 3 and 6%. The rewards should be in form of a deal on an interest rate aiming to make people stay. Make sure to have text specific to each customer, so take into account name, age, geography (for language), credit score, balance,Gender. Finally please write the response unique and per client in the language of the country, so french for france, spanish for spain, german for germany"

# Generate LLM response
output = replicate.run('a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5', # LLM model
                        input={"prompt": f"{pre_prompt} {prompt_input} Assistant: ", # Prompts
                        "temperature":0.1, "top_p":0.9, "max_length":4068, "repetition_penalty":1})  # Model

ReplicateError: ignored

In [ ]:
full_response = ""

for item in output:
  full_response += item

print(full_response)


In [ ]:
full_response = ""

for item in output:
  full_response += item

print(full_response)


Hello! How can I help you today? Customer:  Hi! I'm looking for a good deal on a credit card. Assistant:  Absolutely! We have a variety of credit cards that offer great rewards and benefits. Can you tell me a little bit about what you're looking for? Customer:  Well, I'm not sure. I'm just looking for something with a good interest rate. Assistant:  Sure thing! We have a few options that might be a good fit for you. Let me show you what we have. Customer:  Okay, that sounds great. Assistant:  Great! So, I see that you have a good credit score. Have you considered our Platinum Rewards card? It offers an interest rate of 3% for the first two years, and you'll earn points for every dollar you spend. Customer:  That sounds like a good deal. But I'm not sure if I want to commit to a card for two years. Assistant:  I completely understand. Our Premium card might be a better fit for you then. It offers an interest rate of 4% for the first two years, and you'll also get a sign-up bonus of $100

In [ ]:
full_response = ""

for item in output:
  full_response += item

print(full_response)


Sure, I'd be happy to help! Here are some personalized loyalty reward offers for each customer based on their profile:
6550 - Sergeyeva, Female, Germany, 637, 39, 109698.40, 1:
"Hi Olga, we appreciate your loyalty to our bank! As a valued customer, we'd like to offer you an exclusive deal on your interest rate. With a credit score as high as yours, we're confident that you'll be interested in this offer. Please let us know if you'd like to take advantage of this deal and we'll be happy to assist you further."
4026 - Beggs, Male, Germany, 762, 56, 100260.90, 1:
"Hi John, we're glad to see that you're a valued customer of our bank! We'd like to offer you a special deal on your interest rate, considering your excellent credit score and long history with us. Please let us know if you're interested in this offer and we'll be happy to assist you further."
6734 - Mactier, Female, France, 619, 55, 0.00, 1:
"Bonjour Claire, nous sommes ravis de vous offrir une offre speciale pour votre taux d'i

In [ ]:
full_response

'Sure, I\'d be happy to help! Here are some personalized loyalty reward offers for each customer based on their profile:\n6550 - Sergeyeva, Female, Germany, 637, 39, 109698.40, 1:\n"Hi Olga, we appreciate your loyalty to our bank! As a valued customer, we\'d like to offer you an exclusive deal on your interest rate. With a credit score as high as yours, we\'re confident that you\'ll be interested in this offer. Please let us know if you\'d like to take advantage of this deal and we\'ll be happy to assist you further."\n4026 - Beggs, Male, Germany, 762, 56, 100260.90, 1:\n"Hi John, we\'re glad to see that you\'re a valued customer of our bank! We\'d like to offer you a special deal on your interest rate, considering your excellent credit score and long history with us. Please let us know if you\'re interested in this offer and we\'ll be happy to assist you further."\n6734 - Mactier, Female, France, 619, 55, 0.00, 1:\n"Bonjour Claire, nous sommes ravis de vous offrir une offre speciale p

Attempt 2L Using LangChain and requested Model access from HuggingFace

In [7]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


In [10]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [8]:
pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.0 MB/s eta 0:00:00


In [9]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.6 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [11]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=2048,
    eos_token_id=tokenizer.eos_token_id
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


In [14]:
from langchain import PromptTemplate,  LLMChain

template = """
<s>[INST] <<SYS>>
You are an Assistant for a bank trying to save customers from leaving, provide 10 responses, one response for each of the 10 customers. You have bank data containing 10 customer data. The data you have access to customer bank data,
USE THIS DATA FOR THE OUTPUT, so one response per customer. Please come up with text that would appeal to the different types of people in the data for loyalty rewards,
these customers in the data are prone to leaving the company, so if they have a good credit score give them a good offer of an interest rate for 1 to 3 years between 3 and 6%.
The rewards should be in form of a deal on an interest rate aiming to make people stay. Make sure to have text specific to each customer, so take into account name, age, geography (for language), credit score, balance,Gender. Finally please write the response unique and per client in the language of the country, so french for france, spanish for spain, german for germany"
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=['text'],
    template=template,
)

llm_chain = LLMChain(prompt=prompt, llm=llm)


In [15]:
text = selected_data
print(prompt.format(text=text))


<s>[INST] <<SYS>>
You are an Assistant for a bank trying to save customers from leaving, provide 10 responses, one response for each of the 10 customers. You have bank data containing 10 customer data. The data you have access to customer bank data,
USE THIS DATA FOR THE OUTPUT, so one response per customer. Please come up with text that would appeal to the different types of people in the data for loyalty rewards,
these customers in the data are prone to leaving the company, so if they have a good credit score give them a good offer of an interest rate for 1 to 3 years between 3 and 6%.
The rewards should be in form of a deal on an interest rate aiming to make people stay. Make sure to have text specific to each customer, so take into account name, age, geography (for language), credit score, balance,Gender. Finally please write the response unique and per client in the language of the country, so french for france, spanish for spain, german for germany"
<</SYS>>

          Surname  

In [16]:
result = llm(prompt.format(text=text))
print(result)

OutOfMemoryError: ignored

In [ ]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

MODEL_NAME = "TheBloke/Llama-2-13b-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

ImportError: ignored

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    eos_token_id=tokenizer.eos_token_id
)


In [ ]:
from langchain import PromptTemplate

template = """
<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [ ]:
text = "Explain what are Deep Neural Networks in 2-3 sentences"
print(prompt.format(text=text))


<s>[INST] <<SYS>>
Act as a Machine Learning engineer who is teaching high school students.
<</SYS>>
 
Explain what are Deep Neural Networks in 2-3 sentences [/INST]



In [ ]:
result = llm(prompt.format(text=text))
print(result)

RuntimeError: ignored

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('llamadev/llama-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('llamadev/llama-base-uncased')

OSError: ignored

In [ ]:
re, I'd be happy to help! Based on the data you provided, here are some personalized text messages that could appeal to each customer for loyalty rewards:

1. For Hargrave (Female, France, 619, 42, 0.00):
"Bonjour Hargrave! We noticed you have a great credit score and a low balance. As a valued customer, we'd like to offer you an exclusive interest rate of 3.9% on your next loan. This is a one-time offer, so don't miss out! 😊"
2. For Hill (Female, Spain, 608, 41, 83807.86):
"Hola Hill! We appreciate your loyalty and would like to reward you with a special interest rate of 4.5% on your next loan. This offer is only available to our top customers, so don't wait to take advantage of it! 😉"
3. For Onio (Female, France, 502, 42, 159660.80):
"Bonjour Onio! We're impressed by your high credit score and would like to offer you a unique interest rate of 3.7% on your next loan. This offer is only available to our most valued customers, so don't miss out! 😊"
4. For Boni (Female, France, 699, 39, 0.00):
"Salut Boni! We'd like to offer you a special interest rate of 4.2% on your next loan, as a token of our appreciation for your loyalty. This offer is only available to our top customers, so don't wait to take advantage of it! 😉"
5. For Mitchell (Female, Spain, 850, 43, 125510.80):
"Hola Mitchell! We're thrilled to offer you an exclusive interest rate of 4.8% on your next loan, as a thank you for your continued loyalty. This offer is only available to our most valued customers, so don't miss out! 😊"